In [1]:
import pandas as pd

base = '/Users/maxv/Dropbox (MIT)/inferring_expectations/'

folder = base + 'data/CPUC_fire_reports_raw/'
from os import listdir
files = listdir(folder)
file = files[0]

In [15]:
files

['SCEFire Incident Data Collection2017 Report20180401.xlsx',
 'SCEFire Incident Data Collection2019 Report20200401.xlsx',
 'SDGE CPUC Reportable Ignitions for 2021.xlsx',
 'SDGEFire Incident Data Collection2014 Report20150401.xlsx',
 '.DS_Store',
 'SDGEFire Incident Data Collection2018 Report20190401.xlsx',
 'SDGE CPUC Reportable Ignitions for 2020.xlsx',
 'PGEFire Incident Data Collection2016 Report20170401.xlsx',
 'SDGEFire Incident Data Collection2019 Report20200401.xlsx',
 'SDGEFire Incident Data Collection2017 Report20180401.xlsx',
 'PGEFire Incident Data Collection2015 Report20160401.xlsx',
 'SCEFire Incident Data Collection2014 Report20150401.xlsx',
 'SCEFire Incident Data Collection2018 Report20190401.xlsx',
 '2021_SCE Fire Incident Data Collection Report.xlsx',
 'SDGEFire Incident Data Collection2016 Report20170401.xlsx',
 'PGEFire Incident Data Collection2019 Report20200401.xlsx',
 'SCE Fire Reporting For Year 2020_Final.xlsx',
 'PGEFire Incident Data Collection2017 Report201

In [111]:

overall = None
for file in files:
    if '.xls' not in file: continue
    if 'sdge' in file.lower():
        utility = 'SDG&E'
    elif 'sce' in file.lower():
        utility = 'SCE'
    elif 'pge' in file.lower():
        utility = 'PG&E'
    header_length = 1
    if file == 'SDGEFire Incident Data Collection2016 Report20170401.xlsx':
        header_length = 2
    elif file == 'SCE Fire Reporting For Year 2020_Final.xlsx': 
        header_length = 0
    df = pd.read_excel(folder + file, header=header_length)
    df['Utility'] = utility
    if overall is None: 
        overall = df
    else:
        overall = overall.append(df)
overall = overall.dropna(subset=['Date','Latitude','Longitude'])
def parse_time(text):
#     if text is None: return None
#     if str(text) == 'NaT': return None
# #     if '46:59.5' in text: return None
    text = str(text).split(' ')[0]
    text = text.replace('116','16')
    return parser.parse(text, dayfirst=False)
overall['date'] = overall['Date'].apply(parse_time).dt.date
overall.to_csv(folder + 'CPUC_ignitions.csv', index=False)

In [125]:
pd.read_csv(folder + 'CPUC_ignitions.csv').date.apply(parse_time).dt.date

0       2017-12-21
1       2017-12-05
2       2017-12-01
3       2017-11-13
4       2017-11-11
           ...    
4548    2014-12-19
4549    2014-12-25
4550    2014-12-29
4551    2014-12-30
4552    2014-12-30
Name: date, Length: 4553, dtype: object

In [109]:
def parse_time(text):
    if text is None: return None
    if str(text) == 'NaT': return None
#     if '46:59.5' in text: return None
    text = str(text).split(' ')[0]
    text = text.replace('116','16')
    return parser.parse(text, dayfirst=False)
overall['date'] = overall['Date'].apply(parse_time)
overall.date.dt.date
# overall.to_csv(folder + 'CPUC_ignitions.csv', index=False)

0      2017-12-21
1      2017-12-05
2      2017-12-01
3      2017-11-13
4      2017-11-11
          ...    
249    2014-12-19
250    2014-12-25
251    2014-12-29
252    2014-12-30
253    2014-12-30
Name: date, Length: 4553, dtype: object

In [108]:
overall['date'] = pd.to_datetime(overall['date'])

In [46]:
from shapely.geometry import Point
import geopandas as gpd
import timeit
import numpy as np
clean_name_match = []
max_length = 0 
cpuc = pd.read_csv(folder + 'CPUC_ignitions.csv')
cpuc = cpuc[['Date','Time','Latitude','Longitude','Size','Utility','Facility Identification/Circuit','Circuit']]
def my_filter(string):
    if 'D' in string: 
        string = string.replace('D','')
    if ', ' in string: 
        string = string.replace(', ','')
    return string
cpuc['Latitude'] = cpuc.Latitude.apply(my_filter)
cpuc['Longitude'] = cpuc.Longitude.apply(my_filter)
cpuc = cpuc[cpuc.Latitude != 'Not Provided']
cpuc = cpuc[cpuc.Longitude != 'Not Provided']             
""" 
Read files from all utilities 
"""
ica_lat_lon = gpd.read_file(base + 'data/all_ica_maps/')
ica_lat_lon = ica_lat_lon.to_crs("EPSG:4326")
ica_lat_lon.head()

,clean_name,Utility,geometry
0,abacus,SCE,"MULTILINESTRING ((-117.21004 34.13246, -117.20..."
1,abraham,SCE,"MULTILINESTRING ((-118.78023 34.27865, -118.77..."
2,academy,SCE,"MULTILINESTRING ((-118.89383 34.18550, -118.89..."
3,acadian,SCE,"MULTILINESTRING ((-116.94096 33.72730, -116.94..."
4,acapulco,SCE,"MULTILINESTRING ((-117.85239 33.93461, -117.85..."


In [113]:
cpuc = pd.read_csv(folder + 'CPUC_ignitions.csv')
cpuc = cpuc[['Date','Time','Latitude','Longitude','Size','Utility','Facility Identification/Circuit','Circuit']]
def my_filter(string):
    if 'D' in string: 
        string = string.replace('D','')
    if ', ' in string: 
        string = string.replace(', ','')
    return string
cpuc['Latitude'] = cpuc.Latitude.apply(my_filter)
cpuc['Longitude'] = cpuc.Longitude.apply(my_filter)
cpuc = cpuc[cpuc.Latitude != 'Not Provided']
cpuc = cpuc[cpuc.Longitude != 'Not Provided']

In [112]:
thing = pd.read_csv(folder + 'CPUC_ignitions.csv')
thing[thing.Date =='46:59.5']

,Unnamed: 0,Date,Time,Latitude,Longitude,Material at Origin,Land Use at Origin,Size,Suppressed by,Suppressing Agency,...,Year.1,Revised Suspected Initiating Event,Revised Equipment /Facility Failure,Month,Jan to Aug,Unnamed: 1,Material at Origin - Comments,Notes,EIR Number,date


In [114]:
clean_name_match = []
distance_to_match = []
utility_match = []
for row in cpuc.iterrows():
    row = row[1]
    lon = float(row['Longitude'])
    lat = float(row['Latitude'])
    if not (np.isfinite(lon) and np.isfinite(lat)):
        clean_name_match.append(None)
        continue
    point_1 = Point(lon, lat)
    start_time = timeit.default_timer()
    distances = ica_lat_lon.distance(point_1)
    min_dist = np.min(distances)
    num_matches = sum(distances == min_dist)
    polygon_index = np.argmin(distances)
    max_length = max(max_length, num_matches)
    if num_matches == 1:
        clean_name_match.append(ica_lat_lon.loc[polygon_index]['clean_name'])
        utility_match.append(ica_lat_lon.loc[polygon_index]['Utility'])
    if num_matches == 2:
        name_1 = ica_lat_lon.loc[distances == np.min(distances)].iloc[0]['clean_name']
        name_2 = ica_lat_lon.loc[distances == np.min(distances)].iloc[1]['clean_name']
        utility_match.append(ica_lat_lon.loc[distances == np.min(distances)].iloc[0]['Utility'])
#         Cases like "apple hill 2102; apple hill" where we removed the number from this stub 
        if  (name_1 in name_2):
            clean_name_match.append(name_1)
        elif name_2 in name_1:
            clean_name_match.append(name_2)
        else:
            print(num_matches)
            print(np.arange(len(distances))[distances == np.min(distances)])
            print(timeit.default_timer() - start_time)
            print(row['Circuit'])
            print(ica_lat_lon.loc[distances == np.min(distances)])
            print(ica_lat_lon.loc[distances == np.min(distances)].length)
            clean_name_match.append(None)
    if num_matches > 2:
        print(num_matches)
        print(np.arange(len(distances))[distances == np.min(distances)])
        print(timeit.default_timer() - start_time)
        print(row['Circuit'])
        print(ica_lat_lon.loc[distances == np.min(distances)])
        print(ica_lat_lon.loc[distances == np.min(distances)].length)
        clean_name_match.append(None)
        utility_match.append(None)
    distance_to_match.append(min_dist)
cpuc['closest_circuit'] = clean_name_match
cpuc['distance_to_match'] = distance_to_match
cpuc['utility_match'] = utility_match
cpuc['identifier'] = cpuc['closest_circuit'] + '_' + cpuc['utility_match']
cpuc.to_csv(base + 'data/cpuc_with_circuit.csv', index=False)

/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235

/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235

/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235

/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235

/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235

...

/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235

/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235

/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235

/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235

/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235/2089202958.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = ica_lat_lon.distance(point_1)
/var/folders/10/m_drqhg91g179r68n5cmd1zw0000gn/T/ipykernel_28235

In [132]:
any(['116' in date for date in cpuc.Date])

True

In [64]:
cpuc.to_csv(base + 'data/cpuc_with_circuit.csv', index=False)

In [134]:
def parse_time(text):
#     if text is None: return None
#     if str(text) == 'NaT': return None
# #     if '46:59.5' in text: return None
    text = str(text).split(' ')[0]
    text = text.replace('116','16')
    return parser.parse(text, dayfirst=False)
cpuc['date'] = cpuc.Date.apply(parse_time).dt.date

0       2017-12-21
1       2017-12-05
2       2017-12-01
3       2017-11-13
4       2017-11-11
           ...    
4548    2014-12-19
4549    2014-12-25
4550    2014-12-29
4551    2014-12-30
4552    2014-12-30
Name: Date, Length: 4550, dtype: object

In [129]:
thing = pd.read_csv(base + 'data/cpuc_with_circuit.csv')
thing[~thing.Date.isin(overall.Date.unique())]

,Date,Time,Latitude,Longitude,Size,Utility,Facility Identification/Circuit,Circuit,closest_circuit,distance_to_match,utility_match,identifier
0,2017-12-21 00:00:00,23:40:00,34.179529,-118.133769,Less Than .25 Acres,SCE,NaN,NaN,gorge,0.000025,SCE,gorge_SCE
1,2017-12-05 00:00:00,11:38:00,33.957010,-117.861324,Less Than .25 Acres,SCE,NaN,NaN,brakeman,0.000002,SCE,brakeman_SCE
2,2017-12-01 00:00:00,14:29:00,33.761450,-117.266669,.26 - 9.99 Acres,SCE,NaN,NaN,pawnee,0.000057,SCE,pawnee_SCE
3,2017-11-13 00:00:00,13:49:00,33.739098,-117.277780,Less Than .25 Acres,SCE,NaN,NaN,pawnee,0.000004,SCE,pawnee_SCE
4,2017-11-11 00:00:00,10:32:00,34.427116,-119.644052,Less Than .25 Acres,SCE,NaN,NaN,helena,0.000026,SCE,helena_SCE
...,...,...,...,...,...,...,...,...,...,...,...,...
4545,2014-12-19 13:50:00,2014-12-19 13:50:00,37.460979,-122.176861,< 3 meters,PG&E,NaN,NaN,belle haven,0.000002,PG&E,belle haven_PG&E
4546,2014-12-25 10:50:00,2014-12-25 10:50:00,36.383716,-121.866300,< 3 meters,PG&E,NaN,NaN,otter,0.024421,PG&E,otter_PG&E
4547,2014-12-29 11:58:00,2014-12-29 11:58:00,38.166130,-122.253609,Structure Only,PG&E,NaN,NaN,highway,0.000120,PG&E,highway_PG&E
4548,2014-12-30 09:00:00,2014-12-30 09:00:00,38.405860,-120.515324,< 0.25 Acres,PG&E,NaN,NaN,west point,0.001352,PG&E,west point_PG&E


In [116]:
from dateutil import parser
def parse_time(text):
    if text is None: return None
    if str(text) == 'NaT': return None
    text = text.split(' ')[0]
    return parser.parse(text, dayfirst=False)
cpuc['date_test'] = cpuc['Date'].apply(parse_time)

In [80]:
cpuc['Date'].head().apply(parse_time)

0   2017-12-21
1   2017-12-05
2   2017-12-01
3   2017-11-13
4   2017-11-11
Name: Date, dtype: datetime64[ns]

In [79]:
cpuc.Date.

AttributeError: Can only use .dt accessor with datetimelike values

In [14]:
for file in no_date_files:
    print(file)
    df = pd.read_excel(folder + file, header=0)
    print(df.head())

SDGEFire Incident Data Collection2016 Report20170401.xlsx
     Unnamed: 0           Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  \
0  Utility Name           Fire Start        NaN   Location        NaN   
1           NaN                 Date       Time   Latitude  Longitude   
2         SDG&E  2016-01-03 00:00:00   04:30:00    33.1915  -117.3655   
3         SDG&E  2016-01-09 00:00:00   04:30:00     33.082   -117.091   
4         SDG&E  2016-01-16 00:00:00   14:42:00  32.718764 -116.97455   

           Unnamed: 5          Unnamed: 6           Unnamed: 7     Unnamed: 8  \
0                 NaN                 NaN                 Fire            NaN   
1  Material at Origin  Land Use at Origin                 Size  Suppressed by   
2               Other               Urban  Less Than .25 Acres    Fire Agency   
3          Vegetation               Urban  Less Than .25 Acres    Fire Agency   
4          Vegetation               Urban  Less Than .25 Acres    Fire Agency   

           Unnam

/opt/homebrew/Caskroom/miniforge/base/envs/mypy/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
